In [ ]:
print("Loading Gemma3 4B model...")
from transformers import pipeline
import torch


pipe = pipeline(
    "text2text-generation", 
    model="/scratch/rohank__iitp/gemma3_4b_it",
    device="cuda",
    torch_dtype=torch.bfloat16
)

In [ ]:
def generate(prompt:str) ->str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    # Build prompt
    formatted_input = ""
    for message in messages:
        if isinstance(message["content"], list): 
            for item in message["content"]:
                if item["type"] == "text":
                    formatted_input += f"{message['role']}: {item['text']}\n"
        else:
            formatted_input += f"{message['role']}: {message['content']}\n"
    formatted_input += "assistant:"

    # Generate response
    output = pipe(formatted_input, max_new_tokens=100)[0]["generated_text"]

    # Extract only the assistant's response
    assistant_response = output[len(formatted_input):].strip()

    return assistant_response.strip()


generate("Tell about finance")  # Example usage

In [ ]:
def model_response(dialogue):

    prompt = f"""
Instruction:
Continue the conversation as the insurance agent. Respond appropriately to the latest user message. 
And please be brief.

    Give the reply for this query: {dialogue}
    """
    return generate(prompt)

In [ ]:
import pandas as pd

def create_dataset():

    # Make sure your CSV has the columns: 'conversation_id', 'turn_no', 'utterance', 'new_agent_reply'
    df = pd.read_csv('/home/rohank__iitp/Work/niladri/test_baseline dataset/train_conversation.csv')
    # --- Response Generation and Incremental Saving ---

    if not df.empty:
        output_filename = '/home/rohank__iitp/Work/niladri/test_baseline dataset/gemma/single/gemma_single_dataset.csv'
        header_written = False
        
        # Group by conversation_id to process one conversation at a time
        grouped = df.groupby('conversation_id')

        for conversation_id, group in grouped:
            print(f"\nProcessing Conversation ID: {conversation_id}")
            
            # Ensure the conversation turns are in chronological order
            group = group.sort_values('turn_no')
            conversation_history = ""
            processed_rows = []

            for index, row in group.iterrows():
                # Construct the prompt with the history plus the current user utterance
                sentence = "Conversation History:\n" + conversation_history + "Current Utterance: " + f"User: {row['utterance']}\nAgent:"
                # Your debugging print statements
                print("========================================================================================================================================")
                print(f"Generating for conv_id: {row['conversation_id']}, turn: {row['turn_no']}\nPROMPT:\n{sentence}")
                print("========================================================================================================================================")
                
                
                
                # Generate the response
                '''Change Here😆😆😆😆'''
                qwen_response = model_response(sentence)
                
                
                # Create a dictionary from the original row and add the new response
                current_row_data = row.to_dict()
                current_row_data['Gemma Allexp Response'] = qwen_response
                processed_rows.append(current_row_data)

                # Update the history for the next turn in this conversation
                conversation_history += f"User: {row['utterance']}\nAgent: {row['new_agent_reply']}\n"
            
            # Create a DataFrame for the just-processed conversation
            processed_group_df = pd.DataFrame(processed_rows)

            # Append the processed conversation to the output CSV file
            if not header_written:
                # For the first conversation, write with header and overwrite the file
                processed_group_df.to_csv(output_filename, index=False, mode='w')
                header_written = True
            else:
                # For subsequent conversations, append without the header
                processed_group_df.to_csv(output_filename, index=False, mode='a', header=False)
            
            print(f"Conversation ID {conversation_id} has been processed and saved.")

        print(f"\nProcessing complete. All conversations have been saved to '{output_filename}'")

    else:
        print("\nDataFrame is empty. No responses were generated or saved.")

In [ ]:
from datetime import datetime

print("Starting dataset creation...")
start_time = datetime.now()
print("Started at--->", start_time.strftime('%Y-%m-%d %H:%M:%S'))
create_dataset()
# End timer
end_time = datetime.now()
print("Finished time", end_time.strftime('%Y-%m-%d %H:%M:%S'))

# Print elapsed time
print(f"hey() completed in {end_time - start_time} seconds")